# PoC - Feature Engineering

## Libraries

In [1]:
# Utils
from pprint import pprint

# Preprocessing
from sklearn.preprocessing import RobustScaler

# Data loading and manipulation
import numpy as np
import kagglehub
import pandas as pd

# Visualizations
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt

# Statistics calculations
from scipy.stats import kurtosis

# Encoding
from sklearn.preprocessing import OrdinalEncoder
from category_encoders import HashingEncoder

# Tratamiento de nans
from sklearn.impute import SimpleImputer


/home/druiz35/Documents/SEpsilon/SEpsilon-Proyectos/AI-ML-Data/CreditRiskScore/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [2]:
path = "cleaned_data.csv"
df = pd.read_csv(path)

pd.set_option('display.max_columns', None)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268293 entries, 0 to 268292
Data columns (total 40 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    268293 non-null  float64
 1   funded_amnt                  268293 non-null  float64
 2   funded_amnt_inv              268293 non-null  float64
 3   term                         268293 non-null  object 
 4   int_rate                     268293 non-null  float64
 5   installment                  268293 non-null  float64
 6   sub_grade                    268293 non-null  object 
 7   emp_length                   257576 non-null  object 
 8   home_ownership               268293 non-null  object 
 9   annual_inc                   268293 non-null  float64
 10  verification_status          268293 non-null  object 
 11  loan_status                  268293 non-null  float64
 12  purpose                      268293 non-null  object 
 13 

## Balance de target

In [4]:
loan_status_balance = df["loan_status"].value_counts().to_frame()
loan_status_balance.columns = ["count"]
loan_status_balance["percentage"] = loan_status_balance["count"] / loan_status_balance["count"].sum() * 100
loan_status_balance

,count,percentage
loan_status,,
0.0,209526,78.095962
1.0,58767,21.904038


# Ingeniería de features
Las columnas que se van a dejar para entrenar el modelo deberán seguir los siguientes criterios básicos:
1. Las columnas categóricas no ordinales deberán ser codificadas numéricamente utilizando codificación one-hot. Las ordinales deberán utilizar un encoder ordinal. 

De lo anterior, parece que emp_title, title y url son columnas de texto plano, por lo que se procede a dropearlas.

In [5]:
df = df.drop(columns = ["mths_since_last_major_derog", "mths_since_last_delinq"])

## Separación de columnas numéricas y no numéricas

In [6]:
df_column_types = df.dtypes.to_frame().reset_index()
df_column_types.columns = ["column_name", "type"]

df_num_columns = df_column_types[df_column_types["type"] == "float64"]["column_name"].to_list()
df_object_columns = df_column_types[df_column_types["type"] == "object"]["column_name"].to_list()

df_num = df[df_num_columns]
df_object = df[df_object_columns]

print(f"Numeric columns: {len(df_num_columns)}")
pprint(df_num_columns)
print("\n")

print(f"Non-numeric columns: {len(df_object_columns)}")
pprint(df_object_columns)

Numeric columns: 30
['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'loan_status',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim']


Non-numeric columns: 8
['term',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'verification_status',
 'purpose',
 'addr_state',
 'initial_list_status']


## Variables numéricas

### Imputar valores nulls para variables numéricas

In [7]:
nans = df_num.isnull().sum().to_frame().reset_index()
nans.columns = ["column_name", "count"]
nans["percentage"] = nans["count"] / 268293 * 100
to_impute = nans[nans["count"] > 0]
to_impute

,column_name,count,percentage
8,delinq_2yrs,25,0.009318
9,inq_last_6mths,25,0.009318
10,open_acc,25,0.009318
11,pub_rec,25,0.009318
13,revol_util,243,0.090573
14,total_acc,25,0.009318
25,collections_12_mths_ex_med,139,0.051809
26,acc_now_delinq,25,0.009318
27,tot_coll_amt,66447,24.766580
28,tot_cur_bal,66447,24.766580


In [8]:
columns_to_impute = to_impute["column_name"].to_list()
imputer = SimpleImputer(strategy="mean")
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])
df_num = df[df_num_columns]

In [9]:
nans = df_num.isnull().sum().to_frame().reset_index()
nans.columns = ["column_name", "count"]
nans["percentage"] = nans["count"] / 268293 * 100
nans

,column_name,count,percentage
0,loan_amnt,0,0.0
1,funded_amnt,0,0.0
2,funded_amnt_inv,0,0.0
3,int_rate,0,0.0
4,installment,0,0.0
5,annual_inc,0,0.0
6,loan_status,0,0.0
7,dti,0,0.0
8,delinq_2yrs,0,0.0
9,inq_last_6mths,0,0.0


### Escalar datos

In [10]:
df_num_cols_copy = df_num_columns.copy()
df_num_cols_copy.remove("loan_status")
scaler = RobustScaler()
df[df_num_cols_copy] = scaler.fit_transform(df[df_num_cols_copy])

In [11]:
df[df_num_cols_copy].describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
count,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,268293.000000,2.682930e+05,2.682930e+05
mean,0.144613,0.143255,0.136477,0.037342,0.173555,0.247561,0.029778,0.256687,-0.114043,0.138664,0.146562,0.331311,-0.037527,0.126385,557.263324,557.058445,0.175363,0.174097,0.135402,0.343397,0.883882,151.729324,16.123186,0.302363,0.007216,0.003355,1.000000,3.517053e-17,3.983169e-17
std,0.731592,0.741342,0.751304,0.761817,0.798677,1.391772,0.693514,0.755814,1.154727,0.703483,0.442871,1.491998,0.660428,0.736951,3003.707302,3002.619544,0.783038,0.783807,0.768045,1.113425,6.092981,733.948035,113.943116,0.797122,0.092045,0.063004,86.503843,1.043570e+00,1.742680e+00
min,-1.031390,-1.047836,-1.101149,-1.440415,-1.139742,-1.431048,-1.443463,0.000000,-1.000000,-1.428571,0.000000,-0.824662,-1.490667,-1.375000,0.000000,0.000000,-0.918388,-0.910248,-0.887097,-0.705258,0.000000,0.000000,0.000000,-0.368643,0.000000,0.000000,0.000000,-1.089439e+00,-2.009958e+00
25%,-0.417040,-0.430524,-0.439080,-0.461140,-0.406790,-0.404762,-0.485866,0.000000,-1.000000,-0.428571,0.000000,-0.383372,-0.517333,-0.437500,0.000000,0.000000,-0.419054,-0.421131,-0.451613,-0.367040,0.000000,0.000000,0.000000,-0.320436,0.000000,0.000000,0.000000,-7.875196e-01,-9.283253e-01
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
75%,0.582960,0.569476,0.560920,0.538860,0.593210,0.595238,0.514134,0.000000,0.000000,0.571429,0.000000,0.616628,0.482667,0.562500,0.000000,0.000000,0.580946,0.578869,0.548387,0.632960,0.000000,0.000000,0.000000,0.679564,0.000000,0.000000,1.000000,2.124804e-01,7.167467e-02
max,2.062780,2.095672,2.117241,2.647668,3.446372,205.823381,2.089223,29.000000,32.000000,9.428571,15.000000,131.072642,22.304000,7.937500,35000.000000,35000.000000,3.863311,3.872217,2.338712,11.424799,358.680000,33520.270000,7002.190000,3.623227,6.000000,5.000000,44660.173239,6.246983e+01,1.349379e+02


## Variables categóricas

### Criterio 1: Las columnas categóricas no ordinales deberán ser codificadas numéricamente utilizando codificación one-hot. Las ordinales deberán utilizar un encoder ordinal. Si las columnas categóricas tienen más de 15 valores únicos, dejarlas quietas por ahora.

## 
* Approach: Identificar las columnas categóricas y ordinales para luego convertirlas apropiadamente.

In [13]:
print("VALORES ÚNICOS VARIABLES CATEGÓRICAS")
for col in df_object_columns:
    unique_values = df[col].unique()
    print(f"{col}: {len(unique_values)}")
    print(unique_values)
    print("\n")

VALORES ÚNICOS VARIABLES CATEGÓRICAS
term: 2
[' 36 months' ' 60 months']


sub_grade: 35
['B2' 'C4' 'C5' 'C1' 'A4' 'E1' 'F2' 'B5' 'C3' 'B1' 'D1' 'A1' 'B3' 'B4'
 'C2' 'D2' 'A3' 'A5' 'D5' 'A2' 'E4' 'D3' 'D4' 'F3' 'E3' 'F1' 'E5' 'G4'
 'E2' 'G2' 'G1' 'F5' 'F4' 'G5' 'G3']


emp_length: 12
['10+ years' '< 1 year' '3 years' '9 years' '4 years' '5 years' '1 year'
 '6 years' '2 years' '7 years' '8 years' nan]


home_ownership: 3
['RENT' 'OWN' 'MORTGAGE']


verification_status: 3
['Verified' 'Source Verified' 'Not Verified']


purpose: 14
['credit_card' 'car' 'small_business' 'other' 'wedding'
 'debt_consolidation' 'home_improvement' 'major_purchase' 'medical'
 'moving' 'vacation' 'house' 'renewable_energy' 'educational']


addr_state: 51
['AZ' 'GA' 'IL' 'CA' 'TX' 'VA' 'MO' 'CT' 'UT' 'FL' 'NY' 'PA' 'MN' 'NJ'
 'OR' 'KY' 'OH' 'SC' 'RI' 'LA' 'MA' 'WA' 'WI' 'AL' 'NV' 'AK' 'CO' 'MD'
 'WV' 'VT' 'MI' 'DC' 'SD' 'NC' 'AR' 'NM' 'KS' 'HI' 'OK' 'MT' 'WY' 'NH'
 'DE' 'MS' 'TN' 'IA' 'NE' 'ID' 'IN' 'ME' 'ND']



#### Transformación de variables ordinales

In [14]:
# sub_grade
def sub_grade_mapper(val):
    val = str(val)
    grade_mapping = {
        "A": 7,
        "B": 6,
        "C": 5,
        "D": 4,
        "E": 3,
        "F": 2,
        "G": 1
    }
    num_mapping = {
        "1": 0.8,
        "2": 0.6,
        "3": 0.4,
        "4": 0.2,
        "5": 0.0
    }
    num_val = grade_mapping[val[0]] + num_mapping[val[1]]
    return num_val

df["sub_grade"] = df["sub_grade"].astype(str).apply(sub_grade_mapper)
print(sorted(df["sub_grade"].unique()))


[np.float64(1.0), np.float64(1.2), np.float64(1.4), np.float64(1.6), np.float64(1.8), np.float64(2.0), np.float64(2.2), np.float64(2.4), np.float64(2.6), np.float64(2.8), np.float64(3.0), np.float64(3.2), np.float64(3.4), np.float64(3.6), np.float64(3.8), np.float64(4.0), np.float64(4.2), np.float64(4.4), np.float64(4.6), np.float64(4.8), np.float64(5.0), np.float64(5.2), np.float64(5.4), np.float64(5.6), np.float64(5.8), np.float64(6.0), np.float64(6.2), np.float64(6.4), np.float64(6.6), np.float64(6.8), np.float64(7.0), np.float64(7.2), np.float64(7.4), np.float64(7.6), np.float64(7.8)]


In [15]:
emp_length_nan = df["emp_length"][df["emp_length"].isna()]

In [16]:
# Se puede imputar. Nans cerac al 5%
#df["emp_length"].fillna(df["emp_length"].mode()[0], inplace=True)

# Se crea una categoría especial Unknown
df["emp_length"] = df["emp_length"].fillna("Unknown")
print(df["emp_length"].unique())

['10+ years' '< 1 year' '3 years' '9 years' '4 years' '5 years' '1 year'
 '6 years' '2 years' '7 years' '8 years' 'Unknown']


In [17]:
# Encoding de las variables ordinales

ordered_vals = [
    "Unknown", 
    "< 1 year", 
    "1 year", 
    "2 years", 
    "3 years", 
    "4 years", 
    "5 years", 
    "6 years", 
    "7 years", 
    "8 years", 
    "9 years", 
    "10+ years"
]
ordinal_encoder = OrdinalEncoder(categories=[ordered_vals])
df["emp_length"] = ordinal_encoder.fit_transform(df[["emp_length"]])
print(df["emp_length"].unique())

[11.  1.  4. 10.  5.  6.  2.  7.  3.  8.  9.  0.]


In [18]:
# Onehot encoding de las variables categóricas con poca cardinalidad
categ_cols_onehot = [
    "term",
    "home_ownership",
    "verification_status",
    "initial_list_status",
    "purpose"
]

df = pd.get_dummies(df, columns=categ_cols_onehot)
df

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,sub_grade,emp_length,annual_inc,loan_status,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,-0.627803,-0.637813,-0.643678,-0.519862,-0.660590,6.6,11.0,-0.904762,0.0,AZ,0.999117,0.0,0.0,-1.000000,0.0,0.205920,0.741333,-0.8750,0.0,0.0,-0.433323,-0.427530,-0.426267,-0.246701,0.00,0.00,0.00,-0.349861,0.0,0.0,1.000000,0.000000,0.000000,True,False,False,False,True,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
1,-0.852018,-0.865604,-0.871264,0.278066,-0.995996,5.2,1.0,-0.761905,1.0,GA,-1.355124,0.0,4.0,-1.000000,0.0,-0.697274,-1.240000,-1.1875,0.0,0.0,-0.834907,-0.826753,-0.845027,-0.473511,0.00,117.08,1.11,-0.355547,0.0,0.0,1.000000,0.000000,0.000000,False,True,False,False,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,-0.860987,-0.874715,-0.880460,0.397237,-0.916246,5.0,11.0,-1.184476,0.0,IL,-0.673145,0.0,1.0,-1.142857,0.0,-0.601450,1.136000,-0.8125,0.0,0.0,-0.669804,-0.661624,-0.665899,-0.383788,0.00,0.00,0.00,-0.297517,0.0,0.0,1.000000,0.000000,0.000000,True,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
3,-0.179372,-0.182232,-0.181609,-0.029361,-0.086260,5.8,11.0,-0.304762,0.0,CA,0.323322,0.0,0.0,0.000000,0.0,-0.401948,-0.930667,0.8750,0.0,0.0,0.093467,0.101769,0.034562,0.471307,16.97,0.00,0.00,-0.329520,0.0,0.0,1.000000,0.000000,0.000000,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
4,-0.627803,-0.637813,-0.641379,-0.994819,-0.681456,7.2,4.0,-0.619048,0.0,AZ,-0.454064,0.0,2.0,-0.142857,0.0,-0.223363,-0.736000,-0.6875,0.0,0.0,-0.452332,-0.444117,-0.426267,-0.369021,0.00,0.00,0.00,-0.351019,0.0,0.0,1.000000,0.000000,0.000000,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268288,-0.699552,-0.710706,-0.714943,0.402418,-0.710166,4.6,11.0,-0.333333,1.0,CO,1.818905,0.0,-1.0,0.428571,0.0,0.152684,0.200000,1.3750,0.0,0.0,-0.833474,-0.825320,-0.825534,-0.514576,0.00,0.00,0.00,-0.352485,0.0,0.0,0.000000,0.562887,-0.622203,True,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
268289,-0.109865,-0.111617,-0.110345,-1.317789,-0.123238,7.8,1.0,-0.190476,0.0,FL,-0.275618,1.0,-1.0,-0.142857,0.0,-0.010949,-0.802667,-0.1250,0.0,0.0,-0.002075,0.006212,0.105991,-0.547162,0.00,0.00,0.00,0.664394,0.0,0.0,0.000000,-0.893234,0.826777,True,False,False,False,True,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False
268290,-0.517937,-0.526196,-0.528736,0.488774,-0.473422,4.4,3.0,-0.833333,0.0,FL,0.197880,0.0,0.0,-1.000000,0.0,-0.692064,1.112000,-1.1875,0.0,0.0,

In [19]:
hashing_encoder = HashingEncoder(cols=["addr_state"], n_components=8)
    
hashed = hashing_encoder.fit_transform(df[["addr_state"]])
    
hashed.columns = [f"addr_state_hash_{i}" for i in range(hashed.shape[1])]
    
df = df.drop(columns=["addr_state"]).join(hashed)
df

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,sub_grade,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_hash_0,addr_state_hash_1,addr_state_hash_2,addr_state_hash_3,addr_state_hash_4,addr_state_hash_5,addr_state_hash_6,addr_state_hash_7
0,-0.627803,-0.637813,-0.643678,-0.519862,-0.660590,6.6,11.0,-0.904762,0.0,0.999117,0.0,0.0,-1.000000,0.0,0.205920,0.741333,-0.8750,0.0,0.0,-0.433323,-0.427530,-0.426267,-0.246701,0.00,0.00,0.00,-0.349861,0.0,0.0,1.000000,0.000000,0.000000,True,False,False,False,True,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,0,0,0,0,0,0,1,0
1,-0.852018,-0.865604,-0.871264,0.278066,-0.995996,5.2,1.0,-0.761905,1.0,-1.355124,0.0,4.0,-1.000000,0.0,-0.697274,-1.240000,-1.1875,0.0,0.0,-0.834907,-0.826753,-0.845027,-0.473511,0.00,117.08,1.11,-0.355547,0.0,0.0,1.000000,0.000000,0.000000,False,True,False,False,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,0,1,0,0,0,0,0,0
2,-0.860987,-0.874715,-0.880460,0.397237,-0.916246,5.0,11.0,-1.184476,0.0,-0.673145,0.0,1.0,-1.142857,0.0,-0.601450,1.136000,-0.8125,0.0,0.0,-0.669804,-0.661624,-0.665899,-0.383788,0.00,0.00,0.00,-0.297517,0.0,0.0,1.000000,0.000000,0.000000,True,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,0,0,0,0,0,0,0,1
3,-0.179372,-0.182232,-0.181609,-0.029361,-0.086260,5.8,11.0,-0.304762,0.0,0.323322,0.0,0.0,0.000000,0.0,-0.401948,-0.930667,0.8750,0.0,0.0,0.093467,0.101769,0.034562,0.471307,16.97,0.00,0.00,-0.329520,0.0,0.0,1.000000,0.000000,0.000000,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,0,1,0,0,0,0,0,0
4,-0.627803,-0.637813,-0.641379,-0.994819,-0.681456,7.2,4.0,-0.619048,0.0,-0.454064,0.0,2.0,-0.142857,0.0,-0.223363,-0.736000,-0.6875,0.0,0.0,-0.452332,-0.444117,-0.426267,-0.369021,0.00,0.00,0.00,-0.351019,0.0,0.0,1.000000,0.000000,0.000000,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268288,-0.699552,-0.710706,-0.714943,0.402418,-0.710166,4.6,11.0,-0.333333,1.0,1.818905,0.0,-1.0,0.428571,0.0,0.152684,0.200000,1.3750,0.0,0.0,-0.833474,-0.825320,-0.825534,-0.514576,0.00,0.00,0.00,-0.352485,0.0,0.0,0.000000,0.562887,-0.622203,True,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,1,0,0,0,0,0,0,0
268289,-0.109865,-0.111617,-0.110345,-1.317789,-0.123238,7.8,1.0,-0.190476,0.0,-0.275618,1.0,-1.0,-0.142857,0.0,-0.010949,-0.802667,-0.1250,0.0,0.0,-0.002075,0.006212,0.105991,-0.547162,0.00,0.00,0.00,0.664394,0.0,0.0,0.000000,-0.893234,0.826777,True,False,False,False,True,True,False,False,False

## Guardar dataset después de proceso de feature engineering

In [22]:
df.to_csv("feature_engineered_data.csv", index=False)